In [133]:
import requests
import pandas as pd
import os


response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

city_name = input("Enter name of city: ")
filtered_networks = []
for network in data["networks"]:
    if network["location"]["city"].lower() == city_name.lower():
        filtered_networks.append(network)

if filtered_networks:
    chosen_network = filtered_networks[0]
    chosen_network_id = chosen_network["id"]

    chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
    
    chosen_network_data = chosen_network_response.json()

    bike_stations = chosen_network_data["network"]["stations"]
    
    station_data = []
    for station in bike_stations:
        station_name = station["name"]
        latitude = station["latitude"]
        longitude = station["longitude"]
        total_bikes = station["free_bikes"] + station["empty_slots"]
        
        station_data.append({
            "station_name": station_name,
            "Latitude": latitude,
            "Longitude": longitude,
            "Total Bikes": total_bikes
        })

    if station_data:
        df = pd.DataFrame(station_data)
        print(f"Bike Stations in {city_name} (DataFrame):\n")
        print(df)
    else:
        print("No bike station data available.")
else:
    print(f"No networks found for {city_name}.")
    




Bike Stations in moscow (DataFrame):

                                          station_name   Latitude  Longitude  \
0    ст. м. Кропоткинская (выход к Гоголевскому бул...  55.744901  37.602018   
1    ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково,...  55.633750  37.520553   
2                               ул. Новолесная, д.1/49  55.781240  37.592390   
3    ул. Селезнёвская, д.29, стр.1 (ст. м. Достоевс...  55.781785  37.614473   
4                       ул. Новосущёвская, д.22, стр.8  55.789663  37.604464   
..                                                 ...        ...        ...   
773                               Якиманская наб., д.2  55.739044  37.610879   
774    ул. Снежная, д.21/1 (ст. м. Свиблово, выход №5)  55.854750  37.651250   
775  Щёлковское ш., вл.75 (ст. м. Щёлковская, выход...  55.810920  37.799031   
776                             ул. Партизанская, д.40  55.742889  37.400279   
777                       ст. м. Спортивная (выход №1)  55.721770  37.561740   

 

In [134]:
# view extracted dataframe
print(df.head(3))

                                        station_name   Latitude  Longitude  \
0  ст. м. Кропоткинская (выход к Гоголевскому бул...  55.744901  37.602018   
1  ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково,...  55.633750  37.520553   
2                             ул. Новолесная, д.1/49  55.781240  37.592390   

   Total Bikes  
0           12  
1            6  
2           12  


##### Next we need to combine latitude and longitude into a string separated by ',' so that we can use it to retrieve locations from foursquare API.

In [135]:
# Create a new column concatenating latitude and longitude into a comma-separated string
df['ll'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)

# Use station_name as index column
df.set_index('station_name', inplace=True)

print(df)





                                                     Latitude  Longitude  \
station_name                                                               
ст. м. Кропоткинская (выход к Гоголевскому буль...  55.744901  37.602018   
ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково, ...  55.633750  37.520553   
ул. Новолесная, д.1/49                              55.781240  37.592390   
ул. Селезнёвская, д.29, стр.1 (ст. м. Достоевская)  55.781785  37.614473   
ул. Новосущёвская, д.22, стр.8                      55.789663  37.604464   
...                                                       ...        ...   
Якиманская наб., д.2                                55.739044  37.610879   
ул. Снежная, д.21/1 (ст. м. Свиблово, выход №5)     55.854750  37.651250   
Щёлковское ш., вл.75 (ст. м. Щёлковская, выход №3)  55.810920  37.799031   
ул. Партизанская, д.40                              55.742889  37.400279   
ст. м. Спортивная (выход №1)                        55.721770  37.561740   

           

In [136]:
df.head(3)

,Latitude,Longitude,Total Bikes,ll
station_name,,,,
ст. м. Кропоткинская (выход к Гоголевскому бульвару),55.744901,37.602018,12,"55.7449006,37.6020183"
"ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково, выход №10)",55.633750,37.520553,6,"55.6337498,37.5205529"
"ул. Новолесная, д.1/49",55.781240,37.592390,12,"55.7812396,37.59239"


In [138]:
import requests

# Function to query Foursquare API for restaurants and bars
def search_restaurants_and_bars(ll):
    radius = 1000  # 1000 meters (1 kilometer)
    base_url = f"https://api.foursquare.com/v3/places/search?ll={ll}&query=restaurant,bar&radius={radius}"
    
    try:
        response = requests.get(base_url)
        data = response.json()
        
        if 'response' in data:
            venues = data['response']['venues']
            return venues
        else:
            print("Error: No 'response' in Foursquare API data.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Assuming you have a DataFrame called 'df' with a column 'll_test'
# Iterate through the 'll_test' column and query Foursquare for each location
for ll in df['ll']:
    venues = search_restaurants_and_bars(ll)
    
    if venues:
        # Do something with the retrieved venues (e.g., print or save to a new DataFrame)
        print(venues)
    else:
        print("No venues found for:", ll)


Error: No 'response' in Foursquare API data.
No venues found for: 55.7449006,37.6020183
Error: No 'response' in Foursquare API data.
No venues found for: 55.6337498,37.5205529
Error: No 'response' in Foursquare API data.
No venues found for: 55.7812396,37.59239
Error: No 'response' in Foursquare API data.
No venues found for: 55.7817849,37.6144726
Error: No 'response' in Foursquare API data.
No venues found for: 55.7896632,37.6044645
Error: No 'response' in Foursquare API data.
No venues found for: 55.78921,37.609836
Error: No 'response' in Foursquare API data.
No venues found for: 55.7509180556,37.6005927778
Error: No 'response' in Foursquare API data.
No venues found for: 55.7574181,37.6086772
Error: No 'response' in Foursquare API data.
No venues found for: 55.761072,37.601473
Error: No 'response' in Foursquare API data.
No venues found for: 55.7646115,37.6052356
Error: No 'response' in Foursquare API data.
No venues found for: 55.7667105556,37.6094894444
Error: No 'response' in Fou

KeyboardInterrupt: 

In [ ]:
'''import requests
import pandas as pd

# Foursquare API credentials

# Define a function to search for restaurants and bars
def search_venues(ll, query):
    url = f"https://api.foursquare.com/v3/places/search?ll=" + ll

    try:
        response = requests.get(url)
        data = response.json()
        
        # Check if 'groups' exists in the response
        if 'groups' in data['response']:
            venues = data['response']['groups'][0]['items']
            
            results = []
            for venue in venues:
                venue_name = venue['venue']['name']
                venue_category = venue['venue']['categories'][0]['name']
                results.append({'Name': venue_name, 'Category': venue_category})
            
            return results
        else:
            print("No 'groups' found in the response.")
            return []
    except Exception as e:
        print(f"Error: {e}")
        return []

# Assuming you have a DataFrame called df with a 'll' column containing concatenated lat and lon values
results = []
for index, row in df.iterrows():
    ll = row['ll']  # Assuming 'll' is the name of the column with concatenated lat and lon
    # Query for restaurants
    restaurants = search_venues(ll, 'restaurant')
    
    # Query for bars
    bars = search_venues(ll, 'bar')
    
    results.extend(restaurants)
    results.extend(bars)

# Create a DataFrame from the results
venues_df = pd.DataFrame(results)

# Print or further process the venues DataFrame
print(venues_df.head())'''


'import requests\nimport pandas as pd\n\n# Foursquare API credentials\n\n# Define a function to search for restaurants and bars\ndef search_venues(ll, query):\n    url = f"https://api.foursquare.com/v3/places/search?ll=" + ll\n\n    try:\n        response = requests.get(url)\n        data = response.json()\n        \n        # Check if \'groups\' exists in the response\n        if \'groups\' in data[\'response\']:\n            venues = data[\'response\'][\'groups\'][0][\'items\']\n            \n            results = []\n            for venue in venues:\n                venue_name = venue[\'venue\'][\'name\']\n                venue_category = venue[\'venue\'][\'categories\'][0][\'name\']\n                results.append({\'Name\': venue_name, \'Category\': venue_category})\n            \n            return results\n        else:\n            print("No \'groups\' found in the response.")\n            return []\n    except Exception as e:\n        print(f"Error: {e}")\n        return []\n\

In [ ]:
#####
import requests
import pandas as pd

CLIENT_SECRET = 14H2AZ5ABWGUJFMH4M4ZSNKIQG5PGW3VSKPZJL1CV0RMD005
CLIENT_ID = 5HPSU2X4FJ0QEXJMF5APURJWC4MAZ0HMM55AXHDZCL2E1KKF



 # You can adjust the version as needed
url = f'https://api.foursquare.com/v3/places/search?ll=' + ll
url = f'https://api.foursquare.com/v2/venues/search?ll={ll}&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={API_VERSION}&query={query}'

# create dictionary for headers

headers = {"Accept": "application/json"}
headers['Authorization'] = FOURSQUARE_API_KEY
response = requests.get(url,headers=headers)
data = response.json()
print(data)


'''# Define a function to search for restaurants and bars
def search_venues(ll, query):
    url = f'https://api.foursquare.com/v3/places/search?ll=' + ll
        
    try:
        response = requests.get(url)
        data = response.json()
        venues = data['response']['groups'][0]['items']
        
        results = []
        for venue in venues:
            venue_name = venue['venue']['name']
            venue_category = venue['venue']['categories'][0]['name']
            results.append({'Name': venue_name, 'Category': venue_category})
        
        return results
    except Exception as e:
        print(f"Error: {e}")
        return []

# Assuming you have a DataFrame called df with a 'll' column containing concatenated lat and lon values
results = []
for index, row in df.iterrows():
    ll = row['ll']  # Assuming 'll' is the name of the column with concatenated lat and lon
    # Query for restaurants
    restaurants = search_venues(ll, 'restaurant')
    
    # Query for bars
    bars = search_venues(ll, 'bar')
    
    results.extend(restaurants)
    results.extend(bars)

# Create a DataFrame from the results
venues_df = pd.DataFrame(results)

# Print or further process the venues DataFrame
print(venues_df.head())'''


NameError: name 'FOURSQUARE_API_KEY' is not defined